In [14]:
import sys
import os
from collections import defaultdict

In [15]:
MAPS_DIR = r'steamapps\common\Black Mesa\bshift\maps'
DUMPS_DIR = r'../sourcemod/data/srccoop/dumps'
SCRIPTS_DIR = r'../sourcemod/data/srccoop'

In [16]:
maps = sorted([m[:-4] for m in os.listdir(MAPS_DIR) if '.bsp' in m])

In [17]:
print(maps)

['bs_c0m0a', 'bs_c0m0b', 'bs_c0m0c', 'bs_c1m0a', 'bs_c1m0b', 'bs_c1m0c', 'bs_c2m0a', 'bs_c2m0b', 'bs_c2m0c', 'bs_c2m0d', 'bs_c3m0a', 'bs_c3m0b', 'bs_c3m0c', 'bs_c3m0d', 'bs_c3m1a', 'bs_c3m1b', 'bs_c3m1c', 'bs_c3m1d', 'bs_c3m2a', 'bs_c3m2b', 'bs_c3m2c']


In [18]:
class Entity:
    def __init__(self, data):
        """
        Parses entity information from a dump file.
        """
        self.kv = { v.split('"')[1].strip().lower():v.split('"')[3].strip() for v in data }
        
    def find(self, key):
        """
        Lookup entity info by key.
        """
        key = key.strip().lower()
        return (self.kv[key] if (key in self.kv) else None)
    
class EntityDump:
    def __init__(self, mapList):
        """
        Loads entity dump data for the given maps.
        """
        self.maps = mapList
        self.data = defaultdict(list)
        
    def parseEntityDump(self, mapName, fileContents):
        """
        Extracts each entity from the dump file.
        """
        fileContents = fileContents.strip().replace('\r', '').replace('{', '').replace('\t', '')
        for obj in fileContents.split('}'):
            entityData = obj.strip().split('\n')
            if len(entityData) == 0 or len(entityData[0]) == 0:
                continue
            self.data[mapName].append(Entity(entityData))
    
    def loadEntityDumps(self, path):
        """
        Load entity information from the dump files.
        """
        entityDumpFiles = [file for file in os.listdir(path) if '.txt' in file]
        
        for bspMap in self.maps:
            dumpFile = [f for f in entityDumpFiles if f.startswith(bspMap)]
            
            if len(dumpFile) == 0:
                print('No dump file found for map:', bspMap)
                continue
                
            dumpFile = dumpFile[0] # Load + parse file
            with open('%s/%s' % (path, dumpFile), 'r') as f:
                self.parseEntityDump(bspMap, f.read())

    def findByClassName(self, mapName, key):
        """
        Lookup the desired entities by classname, for the given map.
        """
        if mapName in self.data:
            return [v.kv for v in self.data[mapName] if v.find('classname') == key]
        return None

In [19]:
class EDTScriptBuilder:
    def __init__(self, currentMap, nextMap=None):
        self.currentMap = currentMap
        self.nextMap = nextMap
        
    def write(self, dump, shouldFade=False):
        with open('%s/%s.edt' % (SCRIPTS_DIR, self.currentMap), 'w') as f:
            f.write('#base "base/bshift_sp_campaign.edt"\n')
            f.write('\n')
            
            f.write('"config"\n')
            f.write('{\n')
            
            f.write('\t"chapter" "Blue Shift"\n')
            
            if self.nextMap:
                f.write('\t"nextmap" "%s"\n' % self.nextMap)
                
            if shouldFade:
                f.write('\t"intro_type" "fade"\n')
                
            f.write('\n')
            
            # Entity modify
            f.write('\t"entity"\n')
            f.write('\t{\n')                        
            f.write('\t\t"modify"\n')
            f.write('\t\t{\n')
            f.write('\t\t\t"classname" "logic_auto"\n')            
            f.write('\t\t\t"functions"\n')
            f.write('\t\t\t{\n')
            f.write('\t\t\t\t"delay_output" "OnMapSpawn"\n')
            f.write('\t\t\t\t"delay_output" "OnNewGame"\n')
            f.write('\t\t\t}\n')            
            f.write('\t\t}\n')            
            f.write('\t}\n')
            
            f.write('\n')
            
            # Checkpoints
            mapEntities = dump.findByClassName(self.currentMap, 'info_player_start')
            if len(mapEntities) > 0:
                origin = mapEntities[0].get('origin', None)
                angles = mapEntities[0].get('angles', None)
                
                f.write('\t"checkpoint"\n')
                f.write('\t{\n')                        
                f.write('\t\t"spawn"\n')
                f.write('\t\t{\n')
                f.write('\t\t\t"origin" "%s"\n' % origin)            
                f.write('\t\t\t"angles" "%s"\n' % angles)            
                f.write('\t\t}\n')            
                f.write('\t}\n')
            
            f.write('}')            

In [20]:
dump = EntityDump(maps)
dump.loadEntityDumps(DUMPS_DIR)

In [21]:
for idx, currentMap in enumerate(maps):
    nextMap = maps[idx+1] if ((idx + 1) < len(maps)) else None
    
    bldr = EDTScriptBuilder(currentMap, nextMap)
    bldr.write(dump, (idx == 0))

In [22]:
#dump.findByClassName('bs_c0m0b', 'info_landmark')